# Privacy-QoS Trade-Off Analysis

In [20]:
import os
import pandas as pd
import numpy as np

Define method to get sub-datasets of the main dataset

In [21]:
def getSubDatasets(df):
    firstDf = df[(df["dummyLocation"]==False) & (df["gpsPerturbated"]==True)].reset_index(drop=True)
    secondDf = df[(df["dummyLocation"]==True) & (df["gpsPerturbated"]==False)].reset_index(drop=True)
    thirdDf = df[(df["dummyLocation"]==True) & (df["gpsPerturbated"]==True)].reset_index(drop=True)

    return [firstDf, secondDf, thirdDf]

Import privacy and qos datasets, define alphas

In [22]:
privacyDf = pd.read_csv(os.path.join('csvs', f'average_distances.csv'), index_col=[0])
qosDf = pd.read_csv(os.path.join('csvs', f'qos.csv'), index_col=[0])

CSV_NAMES = [os.path.join('csvs', f'tradeOffComplete.csv'), os.path.join('csvs', f'tradeOff.csv')]

for csv in CSV_NAMES:
    if os.path.exists(csv):
        os.remove(csv)

alphas =  list(map(lambda x: round(x, 5), [*np.arange(0.0, 1.0, 1/500), 1.0]))

privacySubDf = getSubDatasets(privacyDf)
qosSubDf = getSubDatasets(qosDf)

Linear combination to achieve the tradeoff between privacy and qos

In [23]:
def linearCombination(alpha, privacy, qos):
    return alpha * privacy + (1-alpha) * qos

Define min-max scaling function

In [24]:
def minMaxScaling(serie):
    min = np.min(serie)
    max = np.max(serie)

    return (serie - min) / (max - min)


Min-max scaling of privacy and qos values of the main dataset

In [25]:
pValues = privacyDf["avg"]

privacyDf["avg"] = minMaxScaling(privacyDf["avg"])
qosDf["mse"] = minMaxScaling([pow(el, -1) for el in qosDf["mse"]])

privacySD = getSubDatasets(privacyDf)
qosSD = getSubDatasets(qosDf)

Define trade-off getter

In [26]:
def getTradeOff(privacy, qos, csvName):
    df=privacy.copy()
    df.drop('avg', inplace=True, axis=1)
    
    for alpha in alphas:
        df["alpha"]=alpha
        df["tradeOff"]=[linearCombination(alpha, avg, mse) for avg, mse in zip(privacy["avg"], qos["mse"])]
        tradeOff = df[df["tradeOff"]==np.max(df["tradeOff"])]

        if not os.path.exists(csvName):
            tradeOff.to_csv(csvName)
        else:
            tradeOff.to_csv(csvName, mode='a', header=False)

Compute Privacy-QoS trade-off

In [27]:
privacyQos = getTradeOff(privacyDf, qosDf, CSV_NAMES[0])

privacyQosSD = []
for prv, qos in zip(privacySD, qosSD):
    privacyQosSD.append(getTradeOff(prv, qos, CSV_NAMES[1]))

tradeOffComplete = pd.read_csv(CSV_NAMES[0], index_col=0)
tradeOff = pd.read_csv(CSV_NAMES[1], index_col=0)

# In this dataset after the trade-off computation there aren't tradeOff to both true dummy location and true gps perturbated 
tradeOffComplete.reset_index(drop=True)

# print(tradeOffComplete.reset_index(drop=True).to_latex())

,dummyLocation,gpsPerturbated,dumRadMin,dumRadStep,pertDec,alpha,tradeOff
0,False,True,-1,-1,5,0.000,1.000
1,False,True,-1,-1,5,0.002,0.998
2,False,True,-1,-1,5,0.004,0.996
3,False,True,-1,-1,5,0.006,0.994
4,False,True,-1,-1,5,0.008,0.992
...,...,...,...,...,...,...,...
497,True,True,3000,2000,1,0.992,0.992
498,True,True,3000,2000,1,0.994,0.994
499,True,True,3000,2000,1,0.996,0.996
500,True,True,3000,2000,1,0.998,0.998


Only Dummy Location trade-offs of the sub-dataset

In [28]:
tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == False)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == False)].reset_index(drop=True).to_latex())

,dummyLocation,gpsPerturbated,dumRadMin,dumRadStep,pertDec,alpha,tradeOff
0,True,False,500,250,-1,0.000,1.071027e-26
1,True,False,3000,2000,-1,0.002,7.365314e-04
2,True,False,3000,2000,-1,0.004,1.473063e-03
3,True,False,3000,2000,-1,0.006,2.209594e-03
4,True,False,3000,2000,-1,0.008,2.946125e-03
...,...,...,...,...,...,...,...
496,True,False,3000,2000,-1,0.992,3.653196e-01
497,True,False,3000,2000,-1,0.994,3.660561e-01
498,True,False,3000,2000,-1,0.996,3.667926e-01
499,True,False,3000,2000,-1,0.998,3.675291e-01


Only Gps Perturbated trade-offs of the sub-dataset

In [29]:
tradeOff[(tradeOff['dummyLocation'] == False) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == False) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True).to_latex())

,dummyLocation,gpsPerturbated,dumRadMin,dumRadStep,pertDec,alpha,tradeOff
0,False,True,-1,-1,5,0.000,1.000000
1,False,True,-1,-1,5,0.002,0.998000
2,False,True,-1,-1,5,0.004,0.996000
3,False,True,-1,-1,5,0.006,0.994000
4,False,True,-1,-1,5,0.008,0.992000
...,...,...,...,...,...,...,...
496,False,True,-1,-1,1,0.992,0.778176
497,False,True,-1,-1,1,0.994,0.779745
498,False,True,-1,-1,1,0.996,0.781314
499,False,True,-1,-1,1,0.998,0.782882


Both Dummy Location and Gps Perturbated trade-offs of the subdataset

In [30]:
tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True).iloc[:,2:].to_latex(index=False))

,dummyLocation,gpsPerturbated,dumRadMin,dumRadStep,pertDec,alpha,tradeOff
0,True,True,500,250,5,0.000,1.384329e-26
1,True,True,3000,2000,1,0.002,2.000000e-03
2,True,True,3000,2000,1,0.004,4.000000e-03
3,True,True,3000,2000,1,0.006,6.000000e-03
4,True,True,3000,2000,1,0.008,8.000000e-03
...,...,...,...,...,...,...,...
496,True,True,3000,2000,1,0.992,9.920000e-01
497,True,True,3000,2000,1,0.994,9.940000e-01
498,True,True,3000,2000,1,0.996,9.960000e-01
499,True,True,3000,2000,1,0.998,9.980000e-01
